In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from matplotlib import pyplot as plt
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt



In [ ]:
import os

# Define the path to the folder
folder_path = "/content/drive/MyDrive/crack project/Dataset/Negative"

# List of common image file extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff']

# Initialize a counter
image_count = 0

# Check if the folder exists
if not os.path.exists(folder_path):
    print(f"The folder path '{folder_path}' does not exist. Please check the path.")
else:
    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file is an image
        if any(filename.lower().endswith(ext) for ext in image_extensions):
            image_count += 1

    print(f"Number of images in folder: {image_count}")


Number of images in folder: 6020


## Scaling images to get better conversion

In [ ]:
base_dir = '/content/drive/MyDrive/crack project/Dataset'

# Image data generators with validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2  # 20% of the data for validation
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(299, 299),  # InceptionResNetV2 expects (299, 299, 3) input size
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 9853 images belonging to 2 classes.
Found 2463 images belonging to 2 classes.


In [ ]:
from keras.applications import MobileNetV3Large
from keras.models import Sequential, load_model,  Model

In [ ]:
import tensorflow as tf
print(tf.__version__)


2.17.0


In [ ]:
base_model = MobileNetV3Large(weights='imagenet',
                       include_top=False, #the input shape of our images
                       input_shape=(299, 299, 3))

/usr/local/lib/python3.10/dist-packages/keras/src/applications/mobilenet_v3.py:517: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
x = base_model.output
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the InceptionResNetV2 base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Evaluate model before fine-tuning
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation Loss before fine-tuning: {val_loss}')
print(f'Validation Accuracy before fine-tuning: {val_acc}')

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 11/308 ━━━━━━━━━━━━━━━━━━━━ 1:54:04 23s/step - accuracy: 0.4878 - loss: 1.1507

KeyboardInterrupt: 

In [ ]:
model.save('Crack_detection.h5')

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 299, 299, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 299, 299, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv (Conv2D)             │ (None, 150, 150, 16)   │            432 │ rescaling[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv_bn                   │ (None, 150, 150, 16)   │             64 │ conv[0][0]             │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 150, 150, 16)   │              0 │ conv_bn[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 150, 150, 16)   │            144 │ activation[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 150, 150, 16)   │             64 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu (ReLU)              │ (None, 150, 150, 16)   │              0 │ expanded_conv_depthwi… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 150, 150, 16)   │            256 │ re_lu[0][0]            │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_bn  │ (None, 150, 150, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_add (Add)   │ (None, 150, 150, 16)   │              0 │ activation[0][0],      │
│                           │                        │                │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_1_expand    │ (None, 150, 150, 64)   │          1,024 │ expanded_conv_add[0][… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_1_expand_bn │ (None, 150, 150, 64)   │            256 │ expanded_conv_1_expan… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ re_lu_1 (ReLU)            │ (None, 150, 150, 64)   │              0 │ expanded_conv_1_expan… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_1_depthwis… │ (None, 151, 151, 64)   │              0 │ re_lu_1[0][0]          │
│ (ZeroPadding2D)           │                        │                │                        │
├──────────────────────

 Total params: 10,425,221 (39.77 MB)

 Trainable params: 2,476,289 (9.45 MB)

 Non-trainable params: 2,996,352 (11.43 MB)

 Optimizer params: 4,952,580 (18.89 MB)

In [ ]:
# Get true labels and predictions from the training generator
train_generator.reset()
train_predictions = model.predict(train_generator)
train_predicted_classes = np.round(train_predictions).astype(int).flatten()

# Get true labels from the training generator
train_true_classes = train_generator.classes

# Generate confusion matrix for training data
cm_train = confusion_matrix(train_true_classes, train_predicted_classes)

# Plot confusion matrix for training data
plt.figure(figsize=(8, 6))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', xticklabels=train_generator.class_indices.keys(), yticklabels=train_generator.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix (Training Data)')
plt.show()

# Print classification report for training data
report_train = classification_report(train_true_classes, train_predicted_classes, target_names=train_generator.class_indices.keys())
print(report_train)


294/308 ━━━━━━━━━━━━━━━━━━━━ 10s 770ms/step

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])

# Plot training & validation accuracy values
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])

plt.tight_layout()
plt.show()


In [ ]:
from tensorflow.keras.preprocessing import image

def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    class_index = np.round(prediction).astype(int).flatten()[0]
    return 'Crack' if class_index == 1 else 'Not crack'

test_image_path = '/content/istockphoto-173019226-170667a.webp'
prediction = predict_image(model, test_image_path)
print(f'The test image falls into: {prediction}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
The test image falls into: Not crack


# Fine tuning model by unfreeze layers


In [ ]:
# Unfreeze some layers and fine-tune
for layer in base_model.layers[:200]:
    layer.trainable = False
for layer in base_model.layers[200:]:
    layer.trainable = True

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training the model
history_fine = model.fit(train_generator, epochs=10, validation_data=validation_generator)

In [ ]:
val_loss_fine, val_acc_fine = model.evaluate(validation_generator)
print(f'Validation Loss after fine-tuning: {val_loss_fine}')
print(f'Validation Accuracy after fine-tuning: {val_acc_fine}')